In [1]:
# pip install pyscipopt

In [2]:
# !apt-get install -y libgmp-dev libmpfr-dev libmpc-dev
# !pip install pyscipopt


In [7]:
from pyscipopt import Model,quicksum,multidict
import pandas as pd

In [77]:
# def addvars(model,N,L,K,A,S):


# # Ading varaibles ............

#      y = {}
#     for i in N:
#         for j in L:
#             y[i,j] = model.addVar(name = "y(%s,%s)" % (i,j), vtype = 'B')

#     r = {}
#     w = {}
#     z = {}
#     x = {}
     
#     for i in N:
#         for j in K:
#             r[i,j] = model.addVar(name = "r(%s,%s)" % (i,j), lb = 0.0)
           
#     for s in S:
#         for i in N:
#             for j in K:
#                 w[i,j,s] = model.addVar(name = "w(%s,%s,%s)" % (i,j,s), lb = 0.0)
#                 z[i,j,s] = model.addVar(name = "z(%s,%s,%s)" % (i,j,s), lb = 0.0)

       
#        for k in K:
#             for (p,q) in A:
#                 x[p,q,k,s] = model.addVar(name = "x(%s,%s,%s,%s)" % (p,q,k,s), lb = 0.0)

#     return y,r,w,z,x



In [1]:
def addvars(model, I, J, K, L):
    # Adding variables
    x = {}
    for i in I:
        for j in J:
            x[i, j] = model.addVar(name="x(%s,%s)" % (i, j))

    v = {}
    for k in K:
        for j in J:
            v[k, j] = model.addVar(name="v(%s,%s)" % (k, j))

    s = {}
    y = {}
    z = {}
    w = {}
    
    for i in I:
        s[i] = model.addVar(name = "s(%s)"%(i))
    for j in J:
        y[j] = model.addVar(name = "y(%s)"%(j), vtype = 'B')
    for k in K:
        z[k] = model.addVar(name = "z(%s)"%(i), vtype = 'B')     
    for i in I:
        for j in J:
            for l in L:
                w[i, j, l] = model.addVar(name="w(%s,%s,%s)" % (i, j, l), vtype = 'B')
            
    return s, w, x, v, y, z


In [2]:
def addparam(df,I, J, K, L):
    Sk = {}
    popi = {}
    for k in K:
        Sk[k] = df.loc[k-1,'Sk']

    for i in I:
        popi[i] = df.loc[i-1,'popi']
        

    mu = df.loc[0,'mu']
    a = df.loc[0,'a']
    b = df.loc[0,'b']
    T = df.loc[0,'T']
    PI = df.loc[0,'PI']
    phi = df.loc[0,'phi']
    
    Uij = {}
    Bij = {}
    tij = {}
    
    Ukj = {}
    Bkj = {}
    tkj = {}
    
    
    
    Uij_ind = df.columns.get_loc('Uij')
    Bij_ind = df.columns.get_loc('Bij')
    tij_ind = df.columns.get_loc('tij')
    
    Ukj_ind = df.columns.get_loc('Ukj')
    Bkj_ind = df.columns.get_loc('Bkj')
    tkj_ind = df.columns.get_loc('tkj')
    
    for i in I:
        for j in J:
            Uij[i,j] = df.iloc[i-1,Uij_ind + j -1]
            Bij[i,j] = df.iloc[i-1,Bij_ind + j -1]
            tij[i,j] = df.iloc[i-1,tij_ind + j -1]
    
    for k in K:
        for j in J:
            Ukj[k,j] = df.iloc[k-1,Ukj_ind + j -1]
            Bkj[k,j] = df.iloc[k-1,Bkj_ind + j -1]
            tkj[k,j] = df.iloc[k-1,tkj_ind + j -1]
    
    
    
    
    Fk = {}
    Fj = {}
    Qj = {}
    
    for k in K:
        Fk[k] = df.loc[k-1,'Fk']
    for j in J:
        Fj[j] = df.loc[j-1,'Fj']
    for j in J:
        Qj[j] = df.loc[j-1,'Qj']
        
    
    fijl = {}
    delUijl = {}
    
    n = df['I'].count()
    
    fijl_ind = df.columns.get_loc('fijl')
    delUijl_ind = df.columns.get_loc('delUijl')
    
    for i in I:
        for j in J:
            for l in L:
                fijl[i,j,l] = df.iloc[i-1+(l-1)*n,fijl_ind + j -1]
                delUijl[i,j,l] = df.iloc[i-1+(l-1)*n,delUijl_ind + j -1]
          
    
    return Sk,popi,Fj,Fk,Qj,mu,a,b,PI,phi,T,Uij,Bij,tij,Ukj,Bkj,tkj,fijl,delUijl

In [3]:
# def main_model(df,model):
    
#     y,r,w,z,x = addvars(model,N,L,K,A,S)
#     Fij,Ml,tij,Uij,Bij,T,u,a,b,mu,P,qk,del_k,hk,pk,pki,vki = addparam(df, N, L, K, A, S)
#     minobj = model.addVar("minobj")
  
#     for i in N:
#         model.addCons(quicksum(y[i,j] for j in L) <= 1)
#         model.addCons(quicksum(r[i,j] for j in K) <= quicksum(Ml[l]*y[i,l] for l in L))
#     for k in K:
#         for s in S:
#                 model.addCons(quicksum((x[j,i,k,s] - z[i,k,s] + (pki[i,k,s]*r[i,k]/T)) for j in N if (j,i) in A) <= quicksum((x[i,j,k,s] + vki[i,k] - w[i,k,s]) for j in N if (i,j) in A))
#                 model.addCons(quicksum((x[j,i,k,s] - z[i,k,s] + (pki[i,k,s]*r[i,k]/T)) for j in N if (j,i) in A) >= quicksum((x[i,j,k,s] + vki[i,k] - w[i,k,s]) for j in N if (i,j) in A))  
#         obj_exp = quicksum(Fij[i,j]*y[i,j] for j in L)
#         obj_exp += quicksum(qk[k]*r[i,k] for k in K)
#     for s in S:
#         for i in N:
#             obj_exp1 = quicksum(hk[k]*z[i,k]*T)
#             obj_exp1 += quicksum(pk[k]*w[i,k]*T)
#             for j in N:
#                 if (i,j) in A:
#                     obj_exp2 = quicksum(mu*del_k[k]*tij[i,j,s]*(1 + a*((u*quicksum(x[i,j,k] for k in K) + Bij[i,j,s])/Uij[i,j,s])**b)*x[i,j,k,s]*T for k in K)
#         obj _exp += P[s]*(obj_exp2 + obj_exp1)           
        
#     model.addCons(obj_exp <= minobj)
#     model.optimize(minobj, "minimize")
    
#     model.writeProblem("model.lp")         
  

In [4]:
def main_model(df,model):
    
    I,J,K,L = IJKL(df)
    s, w, x, v, y, z = addvars(model,I,J,K,L)
    Sk,popi,Fj,Fk,Qj,mu,a,b,PI,phi,T,Uij,Bij,tij,Ukj,Bkj,tkj,fijl,delUijl = addparam(df, I, J, K, L)
    minobj = model.addVar("minobj")
#     minobj1 = model.addVar("minobj1")
    
#     obj_exp2 = 0.0
    
    for i in I:
            model.addCons(quicksum(x[i, j] for j in J) + s[i] >= popi[i], name="D@S_%s" % i)
            
     
    for j in J:       
           model.addCons(quicksum(x[i, j] for i in I) <= Qj[j]*y[j], name="Shel_Cap_%s" % j)
            
    for k in K:       
           model.addCons(quicksum(v[k, j] for j in J) <= Sk[k]*z[k], name="DC_Cap_%s" % k)
            
    for j in J:       
           model.addCons(quicksum(v[k, j] for k in K) >= phi*quicksum(x[i,j] for i in I), name="En_Rel_at_Shel_%s" % j)
      
    for i in I:       
            for j in J:
                    suw = 0.0
                    for l in L:
                        suw += delUijl[i,j,l]*w[i,j,l]
                        print(suw)
                    model.addCons(x[i,j] <= Uij[i,j] + suw, name="Link_Cap_5a_(%s, %s)" %(i, j))  
    for i in I:
        for j in J:
            for l in L:
                model.addCons(w[i,j,l] <= y[j], name = "Link_Cap_5b_(%s, %s, %s)" %(i, j, l))
    
    
    for i in I:       
            for j in J:
                    model.addCons(quicksum(w[i,j,l] for l in L) <= 1, name="At_most_1_5c_(%s, %s, %s)" %(i, j, l))  
    
    lobj = quicksum(Fj[j]*y[j] for j in J) + quicksum(Fk[k]*z[k] for k in K)
    
    
    for i in I:
        for j in J:
            for l in L:
                lobj += fijl[i,j,l]*w[i,j,l]
    
#     nlobj1 = 0.0;
#     nlobj2 = 0.0;
    for i in I:
        for j in J:
                lobj += mu*tij[i,j]*((1 + a*((x[i,j] + Bij[i,j])/(Uij[i,j] + quicksum(delUijl[i,j,l]*w[i,j,l] for l in L))))**b)*x[i,j]*T
    
    for k in K:
        for j in J:
                lobj += mu*tkj[k,j]*((1 + a*((v[k,j] + Bkj[k,j])/(Ukj[k,j] )))**b)*v[k,j]*T
    
    lobj += PI*quicksum(s[i] for i in I) 
    
#     obj_exp2 = nlobj1 + nlobj2
#     obj_exp = lobj + obj_exp2
    
#     model.addCons(obj_exp2 <= minobj, name = "obj_cons")
    model.addCons(minobj >= lobj, name = "nonlinear_obj_cons")
#     model.addCons(obj_exp2 >= minobj)
#     model.addCons(obj_exp <= minobj1)
#     model.addCons(obj_exp >= minobj1)
    
    model.setObjective(minobj, "minimize")
    model.optimize()
#     print(model.getObjVal())
#     model.writeProblem("model.lp")         
#     sol_value = {}
#     if model.getStatus() == 'optimal':
#         sol_value = {var.name: model.getVal(var) for var in model.getVars()}
    
#     return sol_value

In [5]:
def IJKL(df):  
    I = set()
    L = set()
    K = set()
    J = set()
     
    for v1,v2,v3,v4 in zip(df['I'],df['J'],df['K'],df['L']):
        if pd.notna(v1):
            I.add(int(v1))
        if pd.notna(v2):
            J.add(int(v2))
        if pd.notna(v3):
            K.add(int(v3))   
        if pd.notna(v4):
            L.add(int(v4))
        
#     for value in df['A']:
#         if pd.notna(value):
#             int1 = int(value[1])
#             int2 = int(value[3])
#             tup = (int1, int2)
#             A.add(tup)
#         else: 
#             break
        
    
    return I,J,K,L   

In [8]:
df = pd.read_excel("C:/Users/Divyansh/Desktop/project_fhamid/data/data_init.xlsx")
model = Model("Example2")
# model.redirectOutput() 
# model.printVersion()
main_model(df,model) 


Expr({Term(w(1,1,1)): 6.25, Term(): 0.0})
Expr({Term(w(1,1,1)): 6.25, Term(): 0.0, Term(w(1,1,2)): 4.0})
Expr({Term(w(1,2,1)): 3.0, Term(): 0.0})
Expr({Term(w(1,2,1)): 3.0, Term(): 0.0, Term(w(1,2,2)): 6.25})
Expr({Term(w(1,3,1)): 3.0, Term(): 0.0})
Expr({Term(w(1,3,1)): 3.0, Term(): 0.0, Term(w(1,3,2)): 1.23})
Expr({Term(w(2,1,1)): 3.0, Term(): 0.0})
Expr({Term(w(2,1,1)): 3.0, Term(): 0.0, Term(w(2,1,2)): 6.25})
Expr({Term(w(2,2,1)): 4.98, Term(): 0.0})
Expr({Term(w(2,2,1)): 4.98, Term(): 0.0, Term(w(2,2,2)): 1.23})
Expr({Term(w(2,3,1)): 9.351, Term(): 0.0})
Expr({Term(w(2,3,1)): 9.351, Term(): 0.0, Term(w(2,3,2)): 3.0})
Expr({Term(w(3,1,1)): 4.98, Term(): 0.0})
Expr({Term(w(3,1,1)): 4.98, Term(): 0.0, Term(w(3,1,2)): 3.0})
Expr({Term(w(3,2,1)): 1.865, Term(): 0.0})
Expr({Term(w(3,2,1)): 1.865, Term(): 0.0, Term(w(3,2,2)): 3.0})
Expr({Term(w(3,3,1)): 3.75, Term(): 0.0})
Expr({Term(w(3,3,1)): 3.75, Term(): 0.0, Term(w(3,3,2)): 9.351})
Expr({Term(w(4,1,1)): 1.865, Term(): 0.0})
Expr({Te

In [9]:
model.writeProblem("model_0904.lp")

Exception: SCIP: a required plugin was not found !

In [85]:
model.getObjVal()


92.99999999999996

In [45]:
sol_value = {}
if model.getStatus() == 'optimal':
    sol_value = {var.name: model.getVal(var) for var in model.getVars()}
    
print(sol_value)

{'y(1)': 0.0, 'y(2)': 0.0, 'y(3)': 0.0, 'z(5)': 0.0, 'w(1,1,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(1,2,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(1,3,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(2,1,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(2,2,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(2,3,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(3,1,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(3,2,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(3,3,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(4,1,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(4,2,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(4,3,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(5,1,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(5,2,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'w(5,3,{1: s(1), 2: s(2), 3: s(3), 4: s(4), 5: s(5)})': 0.0, 'xij(3,2)': 0.0, 'xij(3,3)': 0.0

In [48]:
solution_data = {}
for var in model.getVars():
    solution_data[var.name] = model.getVal(var)
solution_data['Objective Value'] = model.getObjVal()
# Convert solution data to a pandas DataFrame
soldf = pd.DataFrame([solution_data])
# solution_df = solution_df.transpose()
soldft = soldf.T

csv_filename = 'solution_pre.csv'
soldft.to_csv(csv_filename, index_label = 'Column')


In [49]:
model.writeProblem("model_2403.lp")

Exception: SCIP: a required plugin was not found !